In [1]:
#Total liquidity, total volume, tx count
import os
import sys
os.path.dirname(sys.executable)

'/opt/anaconda3/bin'

In [3]:
conda install -c conda-forge gql

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - gql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py38h50d1736_0         3.1 MB  conda-forge
    gql-2.0.0                  |     pyh9f0ad1d_0          12 KB  conda-forge
    graphql-core-2.3.2         |     pyh9f0ad1d_0          89 KB  conda-forge
    openssl-1.1.1h             |       haf1e3a3_0         1.9 MB  conda-forge
    promise-2.3                |   py38h50d1

In [4]:
conda install plotly

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - plotly


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py38hecd8cb5_0         2.9 MB
    plotly-4.14.3              |     pyhd3eb1b0_0         3.6 MB
    retrying-1.3.3             |             py_2          14 KB
    ------------------------------------------------------------
                                           Total:         6.5 MB

The following NEW packages will be INSTALLED:

  plotly             pkgs/main/noarch::plotly-4.14.3-pyhd3eb1b0_0
  retrying           pkgs/main/noarch::retrying-1.3.3-py_2

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.10.3-py38h50d173~ --> pkgs/main::conda-4.10.3-py38hecd8cb5_0



plotly-4.14.3        | 3.6 MB    | ################

In [4]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
sample_transport=RequestsHTTPTransport(
   url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
   verify=True,
   retries=5,
)
client = Client(
   transport=sample_transport
)
query = gql('''
query {
   uniswapFactory(id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f",     
block: {number: 11276855}) 
   {
        totalLiquidityUSD,
        totalVolumeUSD,
        txCount
   }
}
''')
response = client.execute(query)
print(response)

Exception: {'locations': [{'line': 2, 'column': 3}], 'message': 'Type `Query` has no field `uniswapFactory`'}

In [24]:
#store into arrays
liquidity=response.get('uniswapFactory').get('totalLiquidityUSD')
volume=response.get('uniswapFactory').get('totalVolumeUSD')
tx=response.get('uniswapFactory').get('txCount')

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
x = np.arange(len(liquidity))
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
   go.Scatter(x=x, y=liquidity, name="Liquidity"),secondary_y=False,
)
fig.add_trace(
   go.Scatter(x=x, y=volume, name="Volume"), secondary_y=True
)
# Add figure title
fig.update_layout(title_text="Uniswap Liquidity vs Volume")
# Set x-axis title
fig.update_xaxes(title_text="Blocks")
# Set y-axes title
fig.update_yaxes(title_text="Liquidity", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)
fig.show()

NameError: name 'liquidity' is not defined

In [8]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

sample_transport=RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)

liquidity = []
volume = []
transactions = []
x = 11276855

while x > 11250000: 
    query = gql('''
    query {
    uniswapFactory(id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f", block: {number: ''' + str(x) + '''}) {
        totalVolumeUSD
        totalLiquidityUSD
        txCount
    }
    }
    ''')
    response = client.execute(query)
    liquidity_value = response['uniswapFactory']['totalLiquidityUSD']
    volume_value = response['uniswapFactory']['totalVolumeUSD']
    transactions_value = response['uniswapFactory']['txCount']
    liquidity.insert(0, liquidity_value)
    volume.append(volume_value)
    transactions.append(transactions_value)
    x=x-1000
    print(x)
print(liquidity)
print(volume)
print(transactions)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# normalize or standardize data as

liquidity = [float(i) for i in liquidity]
volume = [float(i) for i in volume]
transactions = [float(i) for i in transactions]
liquidity = [float(i)/sum(liquidity) for i in liquidity]
volume = [float(i)/sum(volume) for i in volume]
transactions = [float(i)/sum(transactions) for i in transactions]
title = 'Uniswap Liquidity Over Time'
labels = ['Liquidity', 'Volume', 'Transactions']

x = np.arange(len(liquidity))

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=x, y=liquidity, name="Liquidity"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=x, y=volume, name="Volume"),
    secondary_y=True,
)


fig.add_trace(
    go.Scatter(x=x, y=transactions, name="Transactions"),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Uniswap Liquidity vs Volume"
)

# Set x-axis title
fig.update_xaxes(title_text="Blocks")

# Set y-axes titles
fig.update_yaxes(title_text="Liquidity", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)

fig.show()

11275855
11274855
11273855
11272855
11271855
11270855
11269855
11268855
11267855
11266855
11265855
11264855
11263855
11262855
11261855
11260855
11259855
11258855
11257855
11256855
11255855
11254855
11253855
11252855
11251855
11250855
11249855
['3328232968.707095905815088957839271', '3340317189.790134272693096485890519', '3343311115.76620244402073705304284', '3322654891.001432151669985474006094', '3287652062.667633221579972784155343', '3266161285.373393579908784009986331', '3286684309.522150848883781165232213', '3292707859.6511898928082470191976', '3310371966.542373688699035122892458', '3296575637.752871834891902674250602', '3289315582.95131770775678398801662', '3273556685.22045652810339566540544', '3229721389.378540238255445597419812', '3194248968.722995835472160750936321', '3165732164.948185437494642364698308', '3195533754.876350070399691357075604', '3184057537.622968210229389761448174', '3199541216.880352214776142914460972', '3187696482.1860555116251176434976', '3099623851.6987819878

In [15]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

sample_transport=RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)

token0 = []
token1 = []
totalValueLockedToken0 = []
totalValueLockedToken1 = []

x=50
# number_tokens = len(address_count)


while x<51:
    query = gql('''
    query {
    ticks(
      first: 1000
      skip: 0
      where: {poolAddress: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" 
        }
    ) {
    pool
  {
    token0Price,
    token1Price,
    totalValueLockedToken0,
    totalValueLockedToken1,
  }
    }
    }
    ''')
    response = client.execute(query)
    token0Price_value = response['ticks']['totalLiquidityUSD']
    token1Price_value = response['ticks']['totalVolumeUSD']
    totalValueLockedToken0_value = response['ticks']['pool']
    totalValueLockedToken1_value = response['ticks']['pool']
    
    
    token0Price.append(token0Price_value)
    token1Price.append(token1Price_value)
    totalValueLockedToken0.append(totalValueLockedToken0_value)
    totalValueLockedToken1.append(totalValueLockedToken1_value)
    x=x-1
    print(x)
    
print(token0Price)
print(token1Price)
print(totalValuedLockedToken0)
print(totalValuedLockedToken1)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# normalize or standardize data as


# need to transfer to dataframe

token0Price = [float(i) for i in token0Price]
token1Price = [float(i) for i in token1Price]
totalValuedLockedToken0 = [float(i) for i in totalValuedLockedToken0]
token0Price = [float(i)/sum(token0Price) for i in token0Price]
token1Price = [float(i)/sum(token1Price) for i in token1Price]
totalValuedLockedToken0 = [float(i)/sum(totalValuedLockedToken0) for i in totalValuedLockedToken0]
# title = 'Uniswap V3 liquidity data'
# labels = ['token0Price', 'token1Price', 'totalValuedLockedToken0','totalValuedLockedToken1']

# x = np.arange(len(token0Price))

# # Create figure with secondary y-axis
# fig = make_subplots(specs=[[{"secondary_y": True}]])

# # Add traces
# fig.add_trace(
#     go.Scatter(x=x, y=token0Price, name="token0Price"),
#     secondary_y=False,
# )

# fig.add_trace(
#     go.Scatter(x=x, y=token1Price, name="token1Price"),
#     secondary_y=True,
# )


# fig.add_trace(
#     go.Scatter(x=x, y=totalValuedLockedToken0, name="totalValuedLockedToken0"),
#     secondary_y=True,
# )


# # Add figure title
# fig.update_layout(
#     title_text="Uniswap token price vs locked"
# )

# # Set x-axis title
# fig.update_xaxes(title_text="Blocks")

# # Set y-axes titles
# fig.update_yaxes(title_text="Liquidity", secondary_y=False)
# fig.update_yaxes(title_text="Volume", secondary_y=True)

# fig.show()

TypeError: list indices must be integers or slices, not str

In [11]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

sample_transport=RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)
query = gql('''
query {
    ticks(
      first: 1000
      skip: 0
      where: {poolAddress: '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8' 
    }
) {
    pool
    {
    token0Price,
    token1Price,
    totalValueLockedToken0,
    totalValueLockedToken1,
}
}
}
''')
response = client.execute(query)
response

GraphQLSyntaxError: Syntax Error GraphQL request (6:28) Unexpected character "'".

5:       skip: 0
6:       where: {poolAddress: '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8' 
                              ^
7:     }


In [37]:
pip install quantstats

  Created wheel for quantstats: filename=QuantStats-0.0.34-py2.py3-none-any.whl size=37837 sha256=5f51e231b2b6ca3ad7fec911fb3a60c628b661fde6cb767290323b0146007860
  Stored in directory: /Users/GuoJietongPRC/Library/Caches/pip/wheels/36/41/bc/7550d7ad7e0184cd641b2f525bfe6139d83c8acf4986fef5f6
  Created wheel for yfinance: filename=yfinance-0.1.60-py2.py3-none-any.whl size=23809 sha256=b9247d24f637f327995e7313a6983e291d48366cfa7aa1bea96b8e818c92377c
  Stored in directory: /Users/GuoJietongPRC/Library/Caches/pip/wheels/2a/26/37/314a530eb4075da4409fb06b2cb3c293a07d7a6ac3e5f5e633
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8367 sha256=b86ac9a1c821de378495259ecbba3ab1192606085c76ccebd21675004166d214
  Stored in directory: /Users/GuoJietongPRC/Library/Caches/pip/wheels/57/6d/a3/a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built quantstats yfinance multitasking
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip ins

In [38]:
import quantstats as qs
from pycoingecko import CoinGeckoAPI
import pandas as pd 
from datetime import datetime
import statistics
cg = CoinGeckoAPI()
date = []
eth_price = []

eth_price_history = cg.get_coin_market_chart_by_id(id='defipulse-index', vs_currency='usd', days='165')
eth_price_dict = eth_price_history['prices']

for x in eth_price_dict: 
    date.append(datetime.fromtimestamp(x[0]/1000))
    eth_price.append(x[1])
total_yield = .28
daily_reduction = .00105
starting_value = 10000
days = 165
values = []
while days > 0: 
    #print(starting_value)
    starting_value = starting_value + (starting_value*(total_yield/365))
    total_yield = total_yield - daily_reduction
    values.append(starting_value)
    days = days - 1 

list_returns_percent_a = [100 * (b - a) / a for a, b in zip(values[::1], values[1::1])]
print(list_returns_percent_a)
percents_list = statistics.pstdev(list_returns_percent_a)

#fetch the daily returns for a stock
stock = qs.utils.download_returns('FB')
print(stock)
print(type(stock))
# show sharpe ratio
x = pd.Series(list_returns_percent_a)
from datetime import datetime

date_range = pd.date_range(end = datetime.today(), periods = 100).to_pydatetime().tolist()

x.index = date_range
x = x/100
print(x)
qs.plots.rolling_sharpe(x)


x.index = pd.to_datetime(x.index)
print(x)
qs.reports.full(x)

[0.07642465753424639, 0.07613698630136913, 0.07584931506849395, 0.07556164383560765, 0.07527397260274465, 0.07498630136985752, 0.07469863013697778, 0.07441095890410493, 0.07412328767124067, 0.07383561643835089, 0.07354794520547234, 0.07326027397260255, 0.07297260273973467, 0.0726849315068561, 0.07239726027396551, 0.07210958904108963, 0.07182191780821058, 0.07153424657533715, 0.07124657534246823, 0.07095890410959255, 0.07067123287670633, 0.07038356164383112, 0.07009589041096011, 0.0698082191780762, 0.06952054794520617, 0.06923287671233162, 0.06894520547944338, 0.0686575342465779, 0.06836986301369329, 0.06808219178081312, 0.06779452054793852, 0.0675068493150673, 0.06721917808219555, 0.06693150684931924, 0.06664383561643614, 0.06635616438356527, 0.06606849315067825, 0.06578082191780776, 0.06549315068492591, 0.06520547945205298, 0.0649178082191716, 0.06463013698630024, 0.06434246575342543, 0.06405479452054018, 0.06376712328766475, 0.06347945205479709, 0.06319178082191632, 0.062904109589039

ValueError: Length mismatch: Expected axis has 164 elements, new values have 100 elements

In [44]:
from pycoingecko import CoinGeckoAPI
from datetime import datetime
cg = CoinGeckoAPI()
tokens = cg.get_coins_list()
print(tokens)
date = []
eth_price = []
sushi_price = []
dpi_price = [] 
solana_price = []
avalanche_price = []
binance_price = []
terrausd_price = []
sushi_price = []

# retreive JSONs
eth_price_history = cg.get_coin_market_chart_by_id(id='olympus', vs_currency='usd', days='60')
dpi_price_history = cg.get_coin_market_chart_by_id(id='defipulse-index', vs_currency='usd', days='30')

# get prices 
eth_price_dict = eth_price_history['prices']
dpi_price_dict = dpi_price_history['prices']

for x in eth_price_dict: 
    date.append(datetime.fromtimestamp(x[0]/1000))
    eth_price.append(x[1])
for x in dpi_price_dict: 
    date.append(datetime.fromtimestamp(x[0]/1000))
    dpi_price.append(x[1])

print(date)
print(eth_price)

'''
eth_price = eth_price[::24]
dpi_price = dpi_price[::24]
total_yield = .28
daily_reduction = .00105
starting_value = 10000
days = 180 
values = []
while days > 0: 
    print(starting_value)
    starting_value = starting_value + (starting_value*(total_yield/365))
    total_yield = total_yield - daily_reduction
    values.append(starting_value)
    days = days - 1 

print(len(values))

import statistics
list_returns_percent_a = [100 * (b - a) / a for a, b in zip(dpi_price[::1], dpi_price[1::1])]
print(list_returns_percent_a)
print(statistics.pstdev(list_returns_percent_a))
'''
'''
solana_price_history = cg.get_coin_market_chart_by_id(id='defi-pulseindex', vs_currency='usd', days='30')
price_dict = solana_price_history['prices']
for x in price_dict: 
    solana_price.append(x[1])
'''




import plotly.graph_objects as go
from plotly.subplots import make_subplots



fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=date, y=eth_price,
                    mode='lines',
                    name='OHM Price',
                    line_color='blue'))
'''
fig.add_trace(go.Scatter(x=date, y=solana_price,
                    mode='lines',
                    name='SOL Price',yaxis='y2'))
fig.add_trace(go.Scatter(x=date, y=avalanche_price,
                    mode='lines',
                    name='AVAX Price',yaxis='y3'))
fig.add_trace(go.Scatter(x=date, y=binance_price,
                    mode='lines',
                    name='BNB Price',yaxis='y4'))

fig.add_trace(go.Scatter(x=date, y=terrausd_price,
                    mode='lines',
                    name='TerraUSD Price',
                    line_color='blue'))
'''
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linewidth=2,
        zeroline=True,
        linecolor='#F4F4F4',
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=22,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    yaxis2=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    yaxis3=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    yaxis4=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    autosize=True,

    plot_bgcolor='white'
)


fig.add_layout_image(
    dict(
        source="https://images.plot.ly/language-icons/api-home/python-logo.png",
        xref="x",
        yref="y",
        x=0,
        y=3,
        sizex=2,
        sizey=2,
        sizing="stretch",
        opacity=0.5,
        layer="below")
)

fig.show()


[{'id': '01coin', 'symbol': 'zoc', 'name': '01coin'}, {'id': '0-5x-long-algorand-token', 'symbol': 'algohalf', 'name': '0.5X Long Algorand Token'}, {'id': '0-5x-long-altcoin-index-token', 'symbol': 'althalf', 'name': '0.5X Long Altcoin Index Token'}, {'id': '0-5x-long-balancer-token', 'symbol': 'balhalf', 'name': '0.5X Long Balancer Token'}, {'id': '0-5x-long-bitcoin-cash-token', 'symbol': 'bchhalf', 'name': '0.5X Long Bitcoin Cash Token'}, {'id': '0-5x-long-bitcoin-sv-token', 'symbol': 'bsvhalf', 'name': '0.5X Long Bitcoin SV Token'}, {'id': '0-5x-long-bitcoin-token', 'symbol': 'half', 'name': '0.5X Long Bitcoin Token'}, {'id': '0-5x-long-cardano-token', 'symbol': 'adahalf', 'name': '0.5X Long Cardano Token'}, {'id': '0-5x-long-chainlink-token', 'symbol': 'linkhalf', 'name': '0.5X Long Chainlink Token'}, {'id': '0-5x-long-cosmos-token', 'symbol': 'atomhalf', 'name': '0.5X Long Cosmos Token'}, {'id': '0-5x-long-defi-index-token', 'symbol': 'defihalf', 'name': '0.5X Long DeFi Index Toke

[datetime.datetime(2021, 5, 8, 19, 37, 4, 319000), datetime.datetime(2021, 5, 8, 20, 36, 45, 876000), datetime.datetime(2021, 5, 8, 21, 37, 29, 481000), datetime.datetime(2021, 5, 8, 22, 36, 39, 313000), datetime.datetime(2021, 5, 8, 23, 36, 18, 576000), datetime.datetime(2021, 5, 9, 0, 34, 55, 847000), datetime.datetime(2021, 5, 9, 1, 37, 16, 678000), datetime.datetime(2021, 5, 9, 2, 35, 42, 48000), datetime.datetime(2021, 5, 9, 3, 37, 20, 914000), datetime.datetime(2021, 5, 9, 4, 37, 42, 193000), datetime.datetime(2021, 5, 9, 5, 34, 35, 120000), datetime.datetime(2021, 5, 9, 6, 38, 10, 221000), datetime.datetime(2021, 5, 9, 7, 35, 6, 303000), datetime.datetime(2021, 5, 9, 8, 36, 41, 768000), datetime.datetime(2021, 5, 9, 9, 34, 43, 463000), datetime.datetime(2021, 5, 9, 10, 35, 44, 680000), datetime.datetime(2021, 5, 9, 11, 34, 45, 927000), datetime.datetime(2021, 5, 9, 12, 35, 46, 395000), datetime.datetime(2021, 5, 9, 13, 36, 43, 91000), datetime.datetime(2021, 5, 9, 14, 36, 12, 78

In [46]:
import pandas as pd 
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
tokens = cg.get_coins_list()
from datetime import datetime
date = []
# Price History of Yearn DAI deposit 
'''
total_yield = .28
daily_reduction = .00105
starting_value = 10000
days = 180 
values = []
while days > 0: 
    print(starting_value)
    starting_value = starting_value + (starting_value*(total_yield/365))
    total_yield = total_yield - daily_reduction
    values.append(starting_value)
    days = days - 1 

print(len(values))
'''
# Price History of ETH
eth_price = []
eth_price_history = cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='usd', days='179')
eth_price_dict = eth_price_history['prices']
for x in eth_price_dict: 
    date.append(datetime.fromtimestamp(x[0]/1000))
    eth_price.append(15.5*x[1])
#eth_price = eth_price[::24]

# Price history of DPI 
dpi_price = []
dpi_price_history = cg.get_coin_market_chart_by_id(id='defipulse-index', vs_currency='usd', days='179')
dpi_price_dict = dpi_price_history['prices']
for x in dpi_price_dict: 
    date.append(datetime.fromtimestamp(x[0]/1000))
    dpi_price.append(x[1])
#dpi_price = dpi_price[::24]

print(len(dpi_price))

# Price History of Sushiswap strategy 
# 3x in pool value, 300% declining 1.2% daily 
total_yield = 3.4
daily_il_yield = .01777
daily_sushi_yield = .0133
daily_reduction = .00105
starting_value = 10000
days = 180 
values = []
while days > 0: 
    print(starting_value)
    starting_value = starting_value + (starting_value*(daily_sushi_yield)) + (starting_value*(daily_il_yield))
    #total_yield = total_yield - daily_reduction
    values.append(starting_value)
    days = days - 1 

print(len(values))




import statistics

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

print(len(eth_price))
print(len(values))
fig.add_trace(go.Scatter(x=date, y=eth_price,
                    mode='lines',
                    name='ETH Returns',
                    line_color='blue'))
fig.add_trace(go.Scatter(x=date, y=values,
                    mode='lines',
                    name='yvDAI Returns'))
#name='yvDAI Returns',yaxis='y2'))
'''
fig.add_trace(go.Scatter(x=date, y=avalanche_price,
                    mode='lines',
                    name='AVAX Price',yaxis='y3'))
fig.add_trace(go.Scatter(x=date, y=binance_price,
                    mode='lines',
                    name='BNB Price',yaxis='y4'))
fig.add_trace(go.Scatter(x=date, y=terrausd_price,
                    mode='lines',
                    name='TerraUSD Price',
                    line_color='blue'))
'''
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linewidth=2,
        zeroline=True,
        linecolor='#F4F4F4',
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=22,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    yaxis2=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    yaxis3=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    yaxis4=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='#F4F4F4',
        tickfont=dict(
            family='Arial',
            size=22,
            color='blue',
        ),
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    autosize=True,

    plot_bgcolor='white'
)

fig.show()


180
10000
10310.7
10631.053449000001
10961.36027966043
11301.92974354948
11653.080700681563
12015.141918051739
12388.452377445607
12773.361592812842
13170.229937501537
13579.42898165971
14001.341840119878
14436.363531092402
14884.901346003442
15347.375230823769
15824.218179245465
16315.876638074622
16822.8109252196
17345.495660666173
17884.42021084307
18440.089146793965
19013.022716584856
19603.757332389145
20212.846072706474
20840.859200185467
21488.384695535227
22156.02880802551
22844.416623090863
23554.192647570297
24286.021413130304
25040.588098436263
25818.599170654677
26620.783046886918
27447.890776153694
28300.696742568787
29179.9993903604
30086.6219714189
31021.413316070884
31985.248627801207
32979.03030266699
34003.688774170856
35060.18338438434
36149.503282137164
37272.66834911316
38430.730154720106
39624.772940627256
40855.91463589254
42125.30790362972
43434.1412201955
44783.63998790698
46175.067682331246
47609.72703522127
49088.9612542056
50614.155280373765
52186.7370849349